In [24]:
import os
import string
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

## Generating lyrics.csv

In [86]:
genresList = ['blues_lyrics', 'country_lyrics', 'disco_lyrics', 'hiphop_lyrics', 'metal_lyrics', 'pop_lyrics', 'reggae_lyrics', 'rock_lyrics']
genreLyrics = dict()
lyricStopped = {
    'blues': list(),
    'country': list(),
    'disco': list(),
    'hiphop': list(),
    'metal': list(),
    'pop': list(),
    'reggae': list(),
    'rock': list(),
}

header = ['filename', 'lyrics', 'genre']

with open(f'lyrics.csv', 'w', newline = "") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    
for genreName in genresList:
    genre = sorted(os.listdir(f'../genres2/{genreName}'))
    
    for song in genre:
#         print(genreName, song)
        
        if song == '.DS_Store':
            continue
            
        path = f'../genres2/{genreName}/{song}'
        songName = open(path, 'r')
        lyric = songName.read()
        
        with open(f'lyrics.csv', 'a', newline = "") as file:
            writer = csv.writer(file)
            writer.writerow([song, lyric, genreName.replace('_lyrics','')])

In [87]:
dataset = pd.read_csv('lyrics.csv')
X = dataset.iloc[:, 1].values
y = dataset.iloc[:, 2].values

print(X.shape)
print(y.shape)

(646,)
(646,)


In [88]:
lyrics = X[:]
lyrics.shape

(646,)

In [89]:
le = preprocessing.LabelEncoder()
lyric_encoded=le.fit_transform(lyrics.astype(str))
lyric_encoded = np.reshape(lyric_encoded,(-1,1))
lyric_encoded.shape

(646, 1)

In [90]:
features = np.row_stack(lyric_encoded)
print(features.shape)
y.shape

(646, 1)


(646,)

## Training and validating LSTM

In [110]:
from keras.models import Sequential
from keras import layers
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, GlobalMaxPooling1D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder

dataset = pd.read_csv('lyrics.csv')
X = dataset.iloc[:, 1].values
y = dataset.iloc[:, 2].values

encoder = LabelEncoder()
y_enc = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(features, y_enc, test_size = 0.25, random_state = 42)

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(3000, 100, input_length=1, trainable=True))
model.add(LSTM(50, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling1D())
model.add(Dense(8, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

epochs = 20
batch_size = 16

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_test, y_test),callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 484 samples, validate on 162 samples
Epoch 1/20
484/484 [==============================] - 1s 1ms/step - loss: 2.1620 - accuracy: 0.1488 - val_loss: 2.0706 - val_accuracy: 0.1543
Epoch 2/20
484/484 [==============================] - 0s 161us/step - loss: 2.1489 - accuracy: 0.1467 - val_loss: 2.0612 - val_accuracy: 0.1543
Epoch 3/20
484/484 [==============================] - 0s 158us/step - loss: 2.1253 - accuracy: 0.1467 - val_loss: 2.0590 - val_accuracy: 0.1543
Epoch 4/20
484/484 [==============================] - 0s 154us/step - loss: 2.1127 - accuracy: 0.1426 - val_loss: 2.0579 - val_accuracy: 0.1543
Epoch 5/20
484/484 [==============================] - 0s 162us/step - loss: 2.1016 - accuracy: 0.1632 - val_loss: 2.0559 - val_accuracy: 0.1543
Epoch 6/20
484/484 [==============================] - 0s 167us/step - loss: 2.0841 - accuracy: 0.1550 - val_loss: 2.0559 - val_accuracy: 0.1543
Epoch 7/20
484/484 [==============================] - 0s 163us/step - loss: 2.0684 - accurac

In [111]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

162/162 [==============================] - 0s 53us/step
Test set
  Loss: 1.989
  Accuracy: 0.235


## Prediction using trained LSTM

In [112]:
testFile = open('../genres2/rock_lyrics/rock.00031.txt', 'r')
testLyrics = testFile.read().lower()
print(testLyrics)

i met a gin-soaked, bar-room queen in memphis
she tried to take me upstairs for a ride
she had to heave me right across shoulder
'cause i just can't seem to drink you off my mind
it's the honky tonk women
gimme, gimme, gimme the honky tonk blues
i laid a divorcée in new york city
i had to put up some kind of a fight
the lady then she covered me with roses
she blew my nose and then she blew my mind
it's the honky tonk women
gimme, gimme, gimme the honky tonk blues
it's the honky tonk women
gimme, gimme, gimme the honky tonk blues
it's the honky tonk women
gimme, gimme, gimme the honky tonk blues


In [113]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
predictedLabels = []
tk = Tokenizer()
from statistics import mode

labels = ['blues', 'country', 'disco', 'hiphop', 'metal','pop', 'reggae', 'rock']

tk.fit_on_texts(testLyrics)
index_list = tk.texts_to_sequences([testLyrics])
# print(index_list)
padded = pad_sequences(index_list, maxlen=1)
pred = model.predict(padded)
print(pred, labels[np.argmax(pred)])

[[0.04818559 0.04561532 0.04753147 0.09711573 0.05786032 0.04586843
  0.22119156 0.43663153]] rock
